In [1]:
!pip install -q flwr[simulation] torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import FashionMNIST
import time
import flwr as fl

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 1.13.1 and Flower 1.4.0


In [2]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
   process = psutil.Process(os.getpid())
   print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
   print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 124.4 GB  | Proc size: 304.6 MB
GPU RAM Free: 15203MB | Used: 965MB | Util   6% | Total 16376MB


In [3]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5, ))]
    )
    trainset = FashionMNIST("./dataset", train=True, download=True, transform=transform)
    testset = FashionMNIST("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

In [4]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):  # Use the passed 'epochs' variable here
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()  # Make sure to call .item() to get the scalar value
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch}: train loss {epoch_loss:.6f}, accuracy {epoch_acc:.6f}")



def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        epochs = config.get("epochs", 1)
        start_time = time.time()  # Start time measurement
        train(self.net, self.trainloader, epochs)
        training_time = time.time() - start_time  # Calculate duration
        print(f"Training time for Client {self.cid}: {training_time:.2f} seconds")
        return get_parameters(self.net), len(self.trainloader), {"training_time": training_time}



    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [15]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.client_training_times = {}
    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(self, server_round: int, parameters: Parameters, client_manager: ClientManager):
        sample_size, min_num_clients = self.num_fit_clients(client_manager.num_available())
        clients = client_manager.sample(num_clients=sample_size, min_num_clients=min_num_clients)
        epochs_sc = 5
        epochs_hl = 3

        standard_config = {"lr": 0.001, "epochs": epochs_sc}
        higher_lr_config = {"lr": 0.0001, "epochs": epochs_hl}
        fit_configurations = []

        for client in clients:
            # Choose config based on the previous training time
            last_time = self.client_training_times.get(client.cid, 0)  # Default to 0 if no time recorded
            print(f"This is the last time {last_time}")
            


            config_to_use = standard_config if last_time < 13.8 else higher_lr_config
            fit_configurations.append((client, FitIns(parameters, config_to_use)))

        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""
        for client, fit_res in results:
            # Update training times for each client
            self.client_training_times[client.cid] = fit_res.metrics.get("training_time", 0)
        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated


    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.25:
            return []
        config = {"lr": 0.001, "epochs": 2}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evalua
        tion function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [16]:
if DEVICE.type == "cuda":
    # Use a single client to train the global model
    client_resources = {"num_gpus": .25, "num_cpus": 2} 

In [17]:

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,

)

INFO flwr 2024-06-03 16:44:51,134 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)


INFO flwr 2024-06-03 16:44:57,468 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 67747580109.0, 'CPU': 32.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0, 'object_store_memory': 33320391475.0}
INFO flwr 2024-06-03 16:44:57,470 | server.py:86 | Initializing global parameters
INFO flwr 2024-06-03 16:44:57,477 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-06-03 16:44:57,478 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-06-03 16:44:57,480 | server.py:101 | FL starting
DEBUG flwr 2024-06-03 16:44:57,483 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
 pid=9576) [Client 0] fit, config: {'lr': 0.001, 'epochs': 5}
 pid=17512) [Client 4] fit, config: {'lr': 0.001, 'epochs': 5}
 pid=32164) [Client 1] fit, config: {'lr': 0.001, 'epochs': 5}
 pid=34152) [Client 2] fit, config: {'lr': 0.001, 'epochs': 5}
 pid=17512) Epoch 0: train loss 0.037603, accuracy 0.574444
 pid=32164) Epoch 0: train loss 0.037869, accuracy 0.567778
 pid=9576) Epoch 0: train loss 0.037560, accuracy 0.554259
 pid=34152) Epoch 0: train loss 0.037879, accuracy 0.562037
 pid=17512) Epoch 1: train loss 0.021848, accuracy 0.733704
 pid=9576) Epoch 1: train loss 0.022582, accuracy 0.718333
 pid=34152) Epoch 1: train loss 0.022094, accuracy 0.731667
 pid=32164) Epoch 1: train loss 0.022258, accuracy 0.727222
 pid=17512) Epoch 2: train loss

DEBUG flwr 2024-06-03 16:45:57,694 | server.py:232 | fit_round 1 received 10 results and 0 failures
DEBUG flwr 2024-06-03 16:45:57,727 | server.py:168 | evaluate_round 1: strategy sampled 10 clients (out of 10)


 pid=17512) Epoch 4: train loss 0.015886, accuracy 0.807407
 pid=34152) Epoch 4: train loss 0.015522, accuracy 0.814815
 pid=17512) [Client 2] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=34152) [Client 5] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=17512) Client 2 loss 0.01918951099117597
 pid=17512) Client 2 accuracy 0.7866666666666666
 pid=9576) [Client 4] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=34152) Client 5 loss 0.018364514509836834
 pid=34152) Client 5 accuracy 0.7933333333333333
 pid=17512) [Client 1] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=9576) Client 4 loss 0.018304271896680196
 pid=9576) Client 4 accuracy 0.82
 pid=17512) Client 1 loss 0.017778386175632478
 pid=17512) Client 1 accuracy 0.7866666666666666
 pid=17512) [Client 9] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=9576) [Client 3] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=32164) [Client 7] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=34152) [Client 0] evaluate, config

...

DEBUG flwr 2024-06-03 16:47:37,281 | server.py:232 | fit_round 3 received 10 results and 0 failures
DEBUG flwr 2024-06-03 16:47:37,315 | server.py:168 | evaluate_round 3: strategy sampled 10 clients (out of 10)


 pid=32164) Epoch 4: train loss 0.009613, accuracy 0.883889
 pid=32164) [Client 2] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=17512) [Client 8] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=9576) [Client 5] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=32164) Client 2 loss 0.011897010306517283
 pid=32164) Client 2 accuracy 0.8683333333333333
 pid=17512) Client 8 loss 0.010818683455387751
 pid=17512) Client 8 accuracy 0.8933333333333333
 pid=9576) Client 5 loss 0.012944923167427382
 pid=9576) Client 5 accuracy 0.8616666666666667
 pid=32164) [Client 9] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=9576) [Client 1] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=17512) [Client 3] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=34152) [Client 0] evaluate, config: {'lr': 0.001, 'epochs': 2}
 pid=9576) Client 1 loss 0.010142865975697836
 pid=9576) Client 1 accuracy 0.88
 pid=32164) Client 9 loss 0.013020786220828692
 pid=32164) Client 9 accuracy 0.8483333333333334
 pid

DEBUG flwr 2024-06-03 16:47:38,967 | server.py:182 | evaluate_round 3 received 10 results and 0 failures
INFO flwr 2024-06-03 16:47:38,968 | server.py:147 | FL finished in 161.4859661999999
INFO flwr 2024-06-03 16:47:38,970 | app.py:218 | app_fit: losses_distributed [(1, 0.01847034652531147), (2, 0.012714284012715023), (3, 0.01144877314567566)]
INFO flwr 2024-06-03 16:47:38,972 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-06-03 16:47:38,973 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-06-03 16:47:38,974 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-06-03 16:47:38,975 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.01847034652531147
	round 2: 0.012714284012715023
	round 3: 0.01144877314567566

 pid=17512) Client 6 loss 0.010914723351597787
 pid=17512) Client 6 accuracy 0.87


In [9]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=5)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"Client {self.cid} loss {loss}")
        print(f"Client {self.cid} accuracy {accuracy}")
        
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE) #Load Model from here
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [11]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),
    client_resources=client_resources,
)

INFO flwr 2024-06-03 16:37:23,653 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flwr 2024-06-03 16:37:27,385 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 37305393561.0, 'memory': 77045918311.0, 'GPU': 1.0, 'CPU': 32.0, 'node:127.0.0.1': 1.0}
INFO flwr 2024-06-03 16:37:27,388 | server.py:86 | Initializing global parameters
INFO flwr 2024-06-03 16:37:27,390 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2024-06-03 16:37:29,975 | server.py:277 | Received initial parameters from one random client
INFO flwr 2024-06-03 16:37:29,976 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-06-03 16:37:29,978 | server.py:101 | FL starting
DEBUG flwr 2024-06-03 16:37:29,979 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


 pid=33172) [Client 8] get_parameters
 pid=33172) [Client 1] fit, config: {}
 pid=32416) [Client 6] fit, config: {}
 pid=25072) [Client 2] fit, config: {}
 pid=8124) [Client 0] fit, config: {}
 pid=33172) Epoch 0: train loss 0.036626, accuracy 0.565926
 pid=25072) Epoch 0: train loss 0.036225, accuracy 0.570926
 pid=32416) Epoch 0: train loss 0.037493, accuracy 0.565741
 pid=33172) Epoch 1: train loss 0.021394, accuracy 0.737778
 pid=8124) Epoch 0: train loss 0.036356, accuracy 0.564630
 pid=25072) Epoch 1: train loss 0.021136, accuracy 0.738889
 pid=32416) Epoch 1: train loss 0.021538, accuracy 0.736852
 pid=33172) Epoch 2: train loss 0.018254, accuracy 0.775185
 pid=8124) Epoch 1: train loss 0.021554, accuracy 0.725185
 pid=25072) Epoch 2: train loss 0.018483, accuracy 0.770185
 pid=32416) Epoch 2: train loss 0.018837, accuracy 0.773333
 pid=33172) Epoch 3: train loss 0.016549, accuracy 0.798148
 pid=8124) Epoch 2: train loss 0.019377, accuracy 0.758148
 pid=25072) Epoch 3: train los

DEBUG flwr 2024-06-03 16:38:30,790 | server.py:232 | fit_round 1 received 10 results and 0 failures
WARNING flwr 2024-06-03 16:38:30,822 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-06-03 16:38:30,823 | server.py:168 | evaluate_round 1: strategy sampled 10 clients (out of 10)


 pid=25072) Epoch 4: train loss 0.016113, accuracy 0.805370
 pid=25072) [Client 2] evaluate, config: {}
 pid=33172) [Client 8] evaluate, config: {}
 pid=8124) [Client 3] evaluate, config: {}
 pid=32416) [Client 1] evaluate, config: {}
 pid=25072) Client 2 loss 0.019609209249416986
 pid=25072) Client 2 accuracy 0.7983333333333333
 pid=25072) [Client 6] evaluate, config: {}
 pid=33172) Client 8 loss 0.019533524761597316
 pid=33172) Client 8 accuracy 0.795
 pid=32416) Client 1 loss 0.018799980481465656
 pid=32416) Client 1 accuracy 0.7983333333333333
 pid=8124) Client 3 loss 0.019022899866104125
 pid=8124) Client 3 accuracy 0.8016666666666666
 pid=25072) Client 6 loss 0.019640488574902217
 pid=25072) Client 6 accuracy 0.7616666666666667
 pid=8124) [Client 9] evaluate, config: {}
 pid=25072) [Client 5] evaluate, config: {}
 pid=33172) [Client 7] evaluate, config: {}
 pid=32416) [Client 4] evaluate, config: {}
 pid=8124) Client 9 loss 0.020981646180152892
 pid=8124) Client 9 accuracy 0.7566

DEBUG flwr 2024-06-03 16:38:32,523 | server.py:182 | evaluate_round 1 received 10 results and 0 failures
WARNING flwr 2024-06-03 16:38:32,525 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2024-06-03 16:38:32,527 | server.py:218 | fit_round 2: strategy sampled 10 clients (out of 10)


...

DEBUG flwr 2024-06-03 16:40:11,230 | server.py:232 | fit_round 3 received 10 results and 0 failures
DEBUG flwr 2024-06-03 16:40:11,262 | server.py:168 | evaluate_round 3: strategy sampled 10 clients (out of 10)


 pid=8124) Epoch 4: train loss 0.009108, accuracy 0.887963
 pid=8124) [Client 7] evaluate, config: {}
 pid=33172) [Client 2] evaluate, config: {}
 pid=32416) [Client 6] evaluate, config: {}
 pid=8124) Client 7 loss 0.010552470435698827
 pid=8124) Client 7 accuracy 0.87
 pid=33172) Client 2 loss 0.011886837085088094
 pid=33172) Client 2 accuracy 0.86
 pid=32416) Client 6 loss 0.010969374229510625
 pid=32416) Client 6 accuracy 0.8666666666666667
 pid=8124) [Client 5] evaluate, config: {}
 pid=33172) [Client 8] evaluate, config: {}
 pid=8124) Client 5 loss 0.012700791085759798
 pid=8124) Client 5 accuracy 0.8583333333333333
 pid=32416) [Client 0] evaluate, config: {}
 pid=33172) Client 8 loss 0.009708423701425394
 pid=33172) Client 8 accuracy 0.8933333333333333
 pid=33172) [Client 1] evaluate, config: {}
 pid=8124) [Client 3] evaluate, config: {}
 pid=25072) [Client 9] evaluate, config: {}


DEBUG flwr 2024-06-03 16:40:12,778 | server.py:182 | evaluate_round 3 received 10 results and 0 failures
INFO flwr 2024-06-03 16:40:12,779 | server.py:147 | FL finished in 162.79994380000005
INFO flwr 2024-06-03 16:40:12,781 | app.py:218 | app_fit: losses_distributed [(1, 0.01974085352818171), (2, 0.012444209339718024), (3, 0.011011985330531993)]
INFO flwr 2024-06-03 16:40:12,782 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-06-03 16:40:12,783 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-06-03 16:40:12,785 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-06-03 16:40:12,786 | app.py:222 | app_fit: metrics_centralized {}


 pid=32416) Client 0 loss 0.011926309367020925
 pid=32416) Client 0 accuracy 0.8516666666666667
 pid=32416) [Client 4] evaluate, config: {}
 pid=8124) Client 3 loss 0.008706987674037615
 pid=8124) Client 3 accuracy 0.8966666666666666
 pid=33172) Client 1 loss 0.009658037051558495
 pid=33172) Client 1 accuracy 0.8966666666666666
 pid=32416) Client 4 loss 0.010898422574003537
 pid=32416) Client 4 accuracy 0.8716666666666667
 pid=25072) Client 9 loss 0.013112200101216635
 pid=25072) Client 9 accuracy 0.8533333333333334


History (loss, distributed):
	round 1: 0.01974085352818171
	round 2: 0.012444209339718024
	round 3: 0.011011985330531993